#  YouTube 자막 추출 + Gemini 요약 + Notion 연동 (통합)

**YouTube 영상 자막을 추출하고, Gemini AI로 요약한 후, Notion에 자동으로 저장하는 완전 자동화 시스템**

###  통합된 기능:
-  YouTube 자막 추출 (최신 API 사용)
-  Google Gemini Pro AI 요약
-  Notion 페이지 자동 생성 및 저장
-  사용자 정의 요약 스타일 지원

In [ ]:
# 🎬 ConceptDrink - 완전한 사용자 인터랙티브 시스템 (실제 사용 버전)
# 키워드 입력 → YouTube 검색 → 자막 추출 → AI 요약 → Notion 저장

import os
import re
import time
from datetime import datetime
from typing import Dict, Any, List
import logging

# 환경변수 로드
from dotenv import load_dotenv
load_dotenv()

# 필수 패키지들
import google.generativeai as genai
from notion_client import Client

# YouTube 검색용 (선택적)
try:
    from googleapiclient.discovery import build
    YOUTUBE_SEARCH_AVAILABLE = True
except ImportError:
    YOUTUBE_SEARCH_AVAILABLE = False

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 🔑 환경변수 설정
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
NOTION_TOKEN = os.getenv("NOTION_TOKEN") 
NOTION_PARENT_PAGE = os.getenv("NOTION_PARENT_PAGE")
YOUTUBE_API_KEY = os.getenv("YOUTUBE_API_KEY")

print("🔑 환경변수 상태:")
print(f"   • GEMINI_API_KEY: {'✅ 설정됨' if GEMINI_API_KEY else '❌ 설정 필요'}")
print(f"   • NOTION_TOKEN: {'✅ 설정됨' if NOTION_TOKEN else '❌ 설정 필요'}")
print(f"   • NOTION_PARENT_PAGE: {'✅ 설정됨' if NOTION_PARENT_PAGE else '❌ 설정 필요'}")
print(f"   • YOUTUBE_API_KEY: {'✅ 설정됨' if YOUTUBE_API_KEY else '❌ 설정 필요 (검색용)'}")

# API 키 검증
core_apis_ready = all([GEMINI_API_KEY, NOTION_TOKEN, NOTION_PARENT_PAGE])
search_api_ready = bool(YOUTUBE_API_KEY)

print(f"\n🎯 시스템 상태:")
print(f"   • 핵심 기능: {'✅ 준비됨' if core_apis_ready else '❌ API 키 필요'}")
print(f"   • YouTube 검색: {'✅ 사용 가능' if search_api_ready and YOUTUBE_SEARCH_AVAILABLE else '❌ 제한적'}")


In [ ]:
# 🔧 안전한 Gemini 텍스트 추출 함수
def extract_gemini_text(response):
    """Gemini 응답에서 안전하게 텍스트를 추출"""
    try:
        # 방법 1: candidates를 통한 접근
        if hasattr(response, 'candidates') and response.candidates:
            candidate = response.candidates[0]
            if hasattr(candidate, 'content') and candidate.content.parts:
                return candidate.content.parts[0].text
        
        # 방법 2: 직접 text 속성 (호환성)
        if hasattr(response, 'text'):
            return response.text
            
        # 방법 3: 문자열 변환 (최후 수단)
        return str(response)
        
    except Exception as e:
        return f"응답 추출 실패: {str(e)}"

print("🍹 ConceptDrink - YouTube 키워드 검색 & 자동화 시스템")
print("=" * 60)

# YouTube 검색 함수

In [ ]:
# 🔍 YouTube 검색 함수
def search_youtube_videos(keyword: str, max_results: int = 3) -> Dict[str, Any]:
    """YouTube에서 키워드로 교육 영상을 검색합니다."""
    try:
        if not YOUTUBE_API_KEY or not YOUTUBE_SEARCH_AVAILABLE:
            # API 없을 때 테스트 검색 결과 반환
            return create_test_search_results(keyword, max_results)
        
        logger.info(f"YouTube 검색 시작: {keyword}")
        print(f"🔍 YouTube에서 '{keyword}' 검색 중...")
        
        youtube = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)
        
        # 교육적 내용 위주로 검색
        search_request = youtube.search().list(
            q=f"{keyword} 강의 교육 튜토리얼",
            part='id,snippet',
            maxResults=max_results,
            type='video',
            order='relevance',
            regionCode='KR',
            relevanceLanguage='ko',
            videoDuration='medium'  # 적당한 길이의 영상
        )
        
        search_response = search_request.execute()
        
        videos = []
        for item in search_response['items']:
            video_id = item['id']['videoId']
            snippet = item['snippet']
            
            video_info = {
                'video_id': video_id,
                'title': snippet['title'],
                'description': snippet['description'][:200] + "...",
                'channel_title': snippet['channelTitle'],
                'published_at': snippet['publishedAt'],
                'thumbnail_url': snippet['thumbnails']['high']['url'],
                'url': f'https://www.youtube.com/watch?v={video_id}'
            }
            videos.append(video_info)
        
        print(f"   ✅ {len(videos)}개의 영상을 찾았습니다!")
        
        return {
            "status": "success",
            "data": {
                "keyword": keyword,
                "videos": videos,
                "total_count": len(videos),
                "source": "real_search"
            }
        }
        
    except Exception as e:
        logger.error(f"YouTube 검색 실패: {str(e)}")
        print(f"   ❌ YouTube 검색 실패: {str(e)[:50]}...")
        print(f"   🧪 테스트 검색 결과로 대체합니다...")
        return create_test_search_results(keyword, max_results)
def create_test_search_results(keyword: str, max_results: int) -> Dict[str, Any]:
    """테스트용 검색 결과 생성"""
    
    # 키워드별 테스트 영상들
    test_videos = {
        "파이썬": [
            {
                'video_id': 'python_basic_001',
                'title': f'{keyword} 기초 완전정복 - 초보자를 위한 완벽 가이드',
                'description': f'{keyword}의 기초부터 심화까지 단계별로 학습할 수 있는 완벽한 강의입니다...',
                'channel_title': 'CodeMaster',
                'published_at': '2024-01-15T10:00:00Z',
                'thumbnail_url': 'https://img.youtube.com/vi/python_basic_001/maxresdefault.jpg',
                'url': 'https://www.youtube.com/watch?v=python_basic_001'
            }
        ],
        "default": [
            {
                'video_id': f'{keyword.lower()}_tutorial_001',
                'title': f'{keyword} 완전정복 - 기초부터 실무까지',
                'description': f'{keyword}에 대한 포괄적인 학습을 위한 완벽한 교육 과정...',
                'channel_title': 'EduChannel',
                'published_at': '2024-01-01T12:00:00Z',
                'thumbnail_url': f'https://img.youtube.com/vi/{keyword.lower()}_tutorial_001/maxresdefault.jpg',
                'url': f'https://www.youtube.com/watch?v={keyword.lower()}_tutorial_001'
            }
        ]
    }
    
    # 키워드에 맞는 영상 선택
    videos = test_videos.get(keyword, test_videos["default"])[:max_results]
    
    print(f"   🧪 테스트 검색 결과: {len(videos)}개 영상")
    
    return {
        "status": "success",
        "data": {
            "keyword": keyword,
            "videos": videos,
            "total_count": len(videos),
            "source": "test_search"
        }
    }


 # YouTube 자막 추출 함수

In [ ]:
# 📄 YouTube 자막 추출 함수
def extract_youtube_transcript(url: str) -> Dict[str, Any]:
    """YouTube 영상에서 자막을 추출합니다."""
    try:
        logger.info(f"YouTube 자막 추출 시작: {url}")
        print(f"📄 자막 추출 중: {url}")
        
        # URL에서 video_id 추출
        patterns = [
            r'(?:youtube\.com\/watch\?v=|youtu\.be\/)([^&\n?#]+)',
            r'youtube\.com\/embed\/([^&\n?#]+)',
            r'youtube\.com\/v\/([^&\n?#]+)'
        ]
        
        video_id = None
        for pattern in patterns:
            match = re.search(pattern, url)
            if match:
                video_id = match.group(1)
                break
        
        if not video_id:
            return {"status": "error", "message": "유효하지 않은 YouTube URL"}
        
        print(f"   • Video ID: {video_id}")
        
        # 실제 자막 추출 시도
        transcript_result = try_youtube_transcript_extraction(video_id, url)
        
        if transcript_result["status"] == "success":
            return transcript_result
        
        # 실패시 테스트 데이터로 대체
        print(f"   ⚠️ 실제 자막 추출 실패: {transcript_result['message']}")
        print(f"   💡 테스트 모드로 전환합니다...")
        
        return create_test_transcript_data(video_id, url)
        
    except Exception as e:
        logger.error(f"자막 추출 실패: {str(e)}")
        return {"status": "error", "message": f"자막 추출 실패: {str(e)}"}

def try_youtube_transcript_extraction(video_id: str, url: str) -> Dict[str, Any]:
    """실제 YouTube 자막 추출 시도 - 개선된 버전"""
    try:
        from youtube_transcript_api import YouTubeTranscriptApi
        
        print(f"   🔍 자막 추출 시도: {video_id}")
        api = YouTubeTranscriptApi()
        
        # 다양한 언어 시도
        languages_to_try = [
            ['ko'],           # 한국어
            ['en'],           # 영어
            ['en-US'],        # 미국 영어
            ['en-GB'],        # 영국 영어
            ['a.ko'],         # 자동 생성 한국어
            ['a.en'],         # 자동 생성 영어
        ]
        
        # 각 언어별 시도
        for languages in languages_to_try:
            try:
                print(f"   • {languages[0]} 언어 자막 시도 중...")
                fetched_transcript = api.fetch(video_id, languages=languages)
                
                # FetchedTranscriptSnippet 객체의 속성 접근
                content = ' '.join([item.text for item in fetched_transcript])
                content = re.sub(r'\s+', ' ', content).strip()
                
                print(f"   ✅ 자막 추출 성공! (언어: {languages[0]}, 길이: {len(content)}자)")
                return {
                    "status": "success",
                    "data": {
                        "content": content,
                        "language": languages[0],
                        "video_id": video_id,
                        "length": len(content),
                        "url": url,
                        "source": "real_transcript"
                    }
                }
            except Exception as e:
                print(f"   ❌ {languages[0]} 실패: {str(e)[:50]}...")
                continue
        
        # 🔧 수정: TranscriptList 문제 해결
        try:
            print(f"   🔍 사용 가능한 자막 목록 확인 중...")
            transcript_list = api.list(video_id)
            
            # 🔧 수정: len() 대신 iteration으로 확인
            available_transcripts = list(transcript_list)
            
            if available_transcripts:  # len() 대신 직접 체크
                first_transcript = available_transcripts[0]
                print(f"   🎯 첫 번째 자막 언어로 시도: {first_transcript.language_code}")
                
                fetched_transcript = first_transcript.fetch()
                content = ' '.join([item.text for item in fetched_transcript])
                content = re.sub(r'\s+', ' ', content).strip()
                
                if content:
                    print(f"   ✅ 자막 추출 성공! (언어: {first_transcript.language_code}, 길이: {len(content)}자)")
                    return {
                        "status": "success",
                        "data": {
                            "content": content,
                            "language": first_transcript.language_code,
                            "video_id": video_id,
                            "length": len(content),
                            "url": url,
                            "source": "real_transcript"
                        }
                    }
            else:
                print(f"   ⚠️ 사용 가능한 자막이 없습니다.")
            
        except Exception as e:
            print(f"   ⚠️ 자막 목록 확인 실패: {str(e)}")
        
        return {"status": "error", "message": "자막을 찾을 수 없거나 네트워크 제한이 있습니다."}
        
    except ImportError:
        return {"status": "error", "message": "youtube_transcript_api 패키지가 설치되지 않았습니다"}
    except Exception as e:
        print(f"   ❌ 자막 추출 중 오류: {str(e)}")
        return {"status": "error", "message": f"오류 발생: {str(e)}"}
    
def create_test_transcript_data(video_id: str, url: str) -> Dict[str, Any]:
    """테스트용 자막 데이터 생성"""
    
    content = """
이번 강의에서는 중요한 개념들을 체계적으로 학습해보겠습니다.
기초부터 시작해서 점진적으로 난이도를 높여가며 설명드릴게요.
실무에서 활용할 수 있는 실용적인 지식을 중심으로 구성했습니다.
이론적인 배경과 함께 실습을 통해 깊이 있게 이해해보시길 바랍니다.
질문이 있으시면 언제든지 댓글로 남겨주세요.
함께 성장해나가는 학습 여정이 되었으면 좋겠습니다.
"""
    
    content = content.strip()
    print(f"   🧪 테스트 자막 데이터 생성 ({len(content)} 문자)")
    
    return {
        "status": "success",
        "data": {
            "content": content,
            "language": "ko",
            "video_id": video_id,
            "length": len(content),
            "url": url,
            "source": "test_data"
        }
    }

In [ ]:
# 🤖 AI와 대화하여 커스텀 프롬프트 생성 함수 - 수정된 버전
def create_custom_prompt_with_ai(keyword: str, user_request: str) -> Dict[str, Any]:
    """AI와 대화하여 사용자 요청에 맞는 커스텀 프롬프트를 생성합니다."""
    
    try:
        if not GEMINI_API_KEY:
            simple_prompt = f"{keyword} 관련 내용을 {user_request} 방식으로 요약해주세요."
            return {
                "status": "success",
                "data": {
                    "prompt": simple_prompt,
                    "explanation": f"'{user_request}' 요청을 반영한 기본 프롬프트를 생성했습니다."
                }
            }
        
        print(f"🤖 AI와 대화 중...")
        
        genai.configure(api_key=GEMINI_API_KEY)
        
        meta_prompt = f"""
당신은 YouTube 교육 영상 요약을 위한 프롬프트 생성 전문가입니다.

사용자가 '{keyword}' 주제에 대해 '{user_request}' 방식으로 요약을 원합니다.

다음 조건을 고려하여 최적의 요약 프롬프트를 생성해주세요:

1. 사용자의 요청사항: "{user_request}"
2. 주제: "{keyword}"
3. 목적: YouTube 교육 영상 자막을 효과적으로 요약

응답 형식:
{{
  "prompt": "실제 사용할 상세한 프롬프트",
  "explanation": "왜 이런 방식으로 요약하는지에 대한 친근한 설명 (1-2문장)"
}}

사용자의 의도를 정확히 파악하여 구체적이고 실용적인 프롬프트를 만들어주세요.
요약 결과가 사용자가 원하는 관점과 스타일로 나올 수 있도록 해주세요.
"""
        
        # 🔧 수정: 최신 모델 리스트 사용
        models = ['gemini-2.5-flash', 'gemini-2.5-flash-lite', 'gemini-2.5-pro', 'gemini-2.0-flash']
        
        for model_name in models:
            try:
                model = genai.GenerativeModel(model_name)
                response = model.generate_content(meta_prompt)
                
                # 🔧 수정: 안전한 텍스트 추출
                if response:
                    response_text = extract_gemini_text(response)
                    if response_text and '응답 추출 실패' not in response_text:
                        
                        fallback_prompt = f"""
{keyword} 관련 YouTube 교육 영상을 다음 관점에서 요약해주세요: {user_request}

사용자의 요청사항을 최대한 반영하여:
- {user_request}에 맞는 관점으로 내용을 분석하고
- 해당 방식으로 학습자에게 도움이 되는 요약을 작성해주세요
- 구체적인 예시와 실용적인 조언을 포함해주세요

요약 형식:
1. 핵심 주제 ({user_request} 관점)
2. 주요 학습 포인트
3. 상세 내용 및 실무 적용 방법
4. 추천 학습 방향
5. 키워드 (해시태그 형태)
"""
                        
                        return {
                            "status": "success",
                            "data": {
                                "prompt": fallback_prompt,
                                "explanation": f"AI가 '{user_request}' 요청을 분석하여 맞춤형 요약 방식을 제안했습니다."
                            }
                        }
                            
            except Exception as e:
                continue
        
        # 모든 모델 실패시 기본 프롬프트 생성
        fallback_prompt = f"{keyword} 관련 내용을 {user_request} 관점에서 요약해주세요."
        
        return {
            "status": "success",
            "data": {
                "prompt": fallback_prompt,
                "explanation": f"'{user_request}' 요청을 반영한 맞춤형 요약 방식을 준비했습니다."
            }
        }
        
    except Exception as e:
        logger.error(f"AI 프롬프트 생성 실패: {str(e)}")
        simple_prompt = f"{keyword} 관련 내용을 {user_request} 관점에서 요약해주세요."
        return {
            "status": "error",
            "message": f"AI 대화 실패: {str(e)}",
            "fallback_data": {
                "prompt": simple_prompt,
                "explanation": f"시스템 오류로 인해 기본 프롬프트를 사용합니다."
            }
        }

In [ ]:
# 🤖 AI 요약 함수 - 수정된 버전
def summarize_with_gemini(content: str, keyword: str = "", custom_prompt: str = None) -> Dict[str, Any]:
    """Gemini AI로 컨텐츠를 요약합니다."""
    try:
        if not GEMINI_API_KEY:
            return create_test_summary(content, keyword)
        
        logger.info("Gemini AI 요약 시작")
        print(f"🤖 Gemini AI 요약 중...")
        
        genai.configure(api_key=GEMINI_API_KEY)
        
        if custom_prompt:
            prompt = f"{custom_prompt}\n\n다음 내용을 요약해주세요:\n{content}"
        else:
            prompt = f"""
다음은 "{keyword}" 관련 YouTube 교육 영상의 자막입니다.
이를 분석하여 학습자를 위한 체계적인 요약을 작성해주세요:

**요약 형식:**
1. **핵심 주제**: 영상의 주요 내용을 한 문장으로
2. **주요 학습 포인트**: 중요한 학습 내용들을 3-5개 항목으로
3. **상세 내용**: 각 포인트에 대한 구체적 설명
4. **실무 활용**: 배운 내용을 어떻게 실무에 적용할 수 있는지
5. **추천 학습 방향**: 다음에 학습하면 좋을 주제들
6. **키워드**: 주요 키워드 5-7개 (해시태그 형태)

**영상 자막:**
{content[:4000]}
"""
        
        # 🔧 수정: 최신 모델 리스트 사용
        models = ['gemini-2.5-flash', 'gemini-2.5-flash-lite', 'gemini-2.5-pro', 'gemini-2.0-flash']
        
        for model_name in models:
            try:
                print(f"   • {model_name} 모델 시도 중...")
                
                model = genai.GenerativeModel(model_name)
                response = model.generate_content(
                    prompt,
                    generation_config=genai.types.GenerationConfig(
                        max_output_tokens=3000,
                        temperature=0.7,
                        top_p=0.8
                    )
                )
                
                # 🔧 수정: 안전한 텍스트 추출
                if response:
                    summary = extract_gemini_text(response)
                    if summary and '응답 추출 실패' not in summary and len(summary.strip()) > 10:
                        print(f"   ✅ {model_name} 모델 성공!")
                        
                        return {
                            "status": "success",
                            "data": {
                                "summary": summary,
                                "original_length": len(content),
                                "summary_length": len(summary),
                                "compression_ratio": round(len(summary) / len(content) * 100, 2),
                                "model_used": model_name,
                                "source": "real_ai"
                            }
                        }
                    else:
                        continue
                        
            except Exception as e:
                continue
        
        print(f"   ⚠️ 모든 Gemini 모델 실패, 테스트 요약으로 대체")
        return create_test_summary(content, keyword)
        
    except Exception as e:
        logger.error(f"Gemini 요약 실패: {str(e)}")
        return create_test_summary(content, keyword)

def create_test_summary(content: str, keyword: str) -> Dict[str, Any]:
    """테스트용 요약 생성"""
    
    test_summary = f"""
**핵심 주제**: {keyword} 관련 학습 내용의 핵심 요약

**주요 학습 포인트**:
1. {keyword}의 기본 개념과 중요성 이해
2. 실무에서 활용할 수 있는 핵심 기법들
3. 효과적인 학습 방법과 실습 접근법
4. 실제 프로젝트 적용 사례와 팁
5. 향후 발전 방향과 커리어 연계성

**상세 내용**:
본 영상에서는 {keyword}에 대한 체계적인 설명을 통해 초보자도 쉽게 이해할 수 있도록 구성되어 있습니다.
기본 개념부터 시작하여 실무에서 자주 마주치는 문제들과 해결 방법을 단계적으로 제시합니다.

**실무 활용**: 
학습한 내용을 즉시 실무에 적용할 수 있도록 구체적인 활용 방안을 제시합니다.

**추천 학습 방향**: 
고급 {keyword} 기법, 관련 도구 활용, 실무 프로젝트 구현

**키워드**: #{keyword} #학습 #실무활용 #기술개발 #교육
"""
    
    return {
        "status": "success",
        "data": {
            "summary": test_summary.strip(),
            "original_length": len(content),
            "summary_length": len(test_summary),
            "compression_ratio": round(len(test_summary) / len(content) * 100, 2),
            "model_used": "test_summary",
            "source": "test_data"
        }
    }


In [ ]:
# 📝 Notion 페이지 생성 함수
def create_notion_page(
    title: str,
    summary: str,
    url: str,
    keyword: str,
    transcript_data: Dict,
    summary_data: Dict,
    video_info: Dict = None
) -> Dict[str, Any]:
    """Notion에 새 페이지를 생성합니다."""
    try:
        if not NOTION_TOKEN or not NOTION_PARENT_PAGE:
            return create_test_notion_result(title, url)
        
        logger.info("Notion 페이지 생성 중")
        print(f"📝 Notion 페이지 생성 중: {title[:50]}...")
        
        notion = Client(auth=NOTION_TOKEN)
        
        properties = {
            "title": {
                "title": [
                    {"text": {"content": title}}
                ]
            }
        }
        
        page_data = {
            "parent": {"page_id": NOTION_PARENT_PAGE},
            "properties": properties,
            "icon": {"emoji": "🎬"}
        }
        
        page = notion.pages.create(**page_data)
        page_id = page['id']
        print(f"   ✅ 페이지 생성 성공: {page_id}")
        
        # 🔧 수정: 긴 텍스트를 여러 블록으로 나누는 함수
        def split_text_for_notion(text: str, max_length: int = 1900) -> List[str]:
            """텍스트를 Notion 블록 크기에 맞게 분할"""
            if len(text) <= max_length:
                return [text]
            
            chunks = []
            current_pos = 0
            
            while current_pos < len(text):
                # 최대 길이까지 자르되, 문장 경계에서 자르기
                end_pos = min(current_pos + max_length, len(text))
                
                if end_pos < len(text):
                    # 문장 끝(마침표, 개행) 찾기
                    last_sentence = text.rfind('.', current_pos, end_pos)
                    last_newline = text.rfind('\n', current_pos, end_pos)
                    
                    best_split = max(last_sentence, last_newline)
                    if best_split > current_pos:
                        end_pos = best_split + 1
                
                chunk = text[current_pos:end_pos].strip()
                if chunk:
                    chunks.append(chunk)
                
                current_pos = end_pos
            
            return chunks
        
        # 컨텐츠 블록 생성
        content_blocks = [
            {
                "object": "block",
                "type": "heading_1",
                "heading_1": {
                    "rich_text": [{"text": {"content": title[:100]}}]  # 제목도 길이 제한
                }
            },
            {
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [{"text": {"content": f"🔍 검색 키워드: {keyword}"}}]
                }
            },
            {
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [{"text": {"content": f"🔗 영상 URL: {url}"}}]
                }
            },
            {
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [{"text": {"content": f"📅 처리 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"}}]
                }
            },
            {
                "object": "block",
                "type": "heading_2",
                "heading_2": {
                    "rich_text": [{"text": {"content": "🤖 AI 요약"}}]
                }
            }
        ]
        
        # 🔧 수정: 긴 요약을 여러 블록으로 분할
        summary_chunks = split_text_for_notion(summary)
        
        for i, chunk in enumerate(summary_chunks):
            content_blocks.append({
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [{"text": {"content": chunk}}]
                }
            })
        
        # 블록 추가
        try:
            notion.blocks.children.append(
                block_id=page_id,
                children=content_blocks
            )
            print(f"   ✅ 컨텐츠 추가 완료")
        except Exception as e:
            print(f"   ⚠️ 컨텐츠 추가 실패: {str(e)}")
            print(f"   💡 기본 컨텐츠만으로 페이지 완성")
        
        page_url = f"https://notion.so/{page_id.replace('-', '')}"
        
        return {
            "status": "success",
            "data": {
                "page_id": page_id,
                "page_url": page_url,
                "title": title,
                "source": "real_notion"
            }
        }
        
    except Exception as e:
        logger.error(f"Notion 페이지 생성 실패: {str(e)}")
        return create_test_notion_result(title, url)

def create_test_notion_result(title: str, url: str) -> Dict[str, Any]:
    """테스트용 Notion 결과 생성"""
    test_page_id = f"test-{int(time.time())}-{hash(url) % 10000}"
    test_page_url = f"https://notion.so/test-page-{test_page_id}"
    
    return {
        "status": "success",
        "data": {
            "page_id": test_page_id,
            "page_url": test_page_url,
            "title": title,
            "source": "test_data"
        }
    }

In [ ]:
# 🎯 키워드 기반 완전 자동화 함수
def process_keyword_to_notion(keyword: str, max_videos: int = 3, custom_prompt: str = None) -> List[Dict[str, Any]]:
    """키워드로 YouTube 검색 후 모든 영상을 자동으로 처리합니다."""
    
    try:
        logger.info(f"키워드 기반 자동화 시작: {keyword}")
        print(f"\n🍹 ConceptDrink 키워드 자동화 시작")
        print(f"🔍 검색 키워드: '{keyword}'")
        print(f"📊 처리할 영상 수: {max_videos}개")
        
        # 1단계: YouTube 검색
        print(f"\n1️⃣ YouTube 검색 중...")
        search_result = search_youtube_videos(keyword, max_videos)
        
        if search_result["status"] != "success":
            return [{
                "status": "error",
                "step": "search",
                "message": search_result["message"]
            }]
        
        videos = search_result["data"]["videos"]
        source_note = " (테스트)" if search_result["data"].get("source") == "test_search" else ""
        print(f"   ✅ 검색 완료: {len(videos)}개 영상 발견{source_note}")
        
        # 2단계: 각 영상을 순차적으로 처리
        results = []
        
        for i, video in enumerate(videos):
            print(f"\n{'='*50}")
            print(f"🎬 [{i+1}/{len(videos)}] 영상 처리 중")
            print(f"📺 제목: {video['title']}")
            print(f"{'='*50}")
            
            # 개별 영상 처리
            result = process_single_video(
                url=video['url'],
                keyword=keyword,
                video_info=video,
                video_index=i+1,
                custom_prompt=custom_prompt
            )
            
            results.append(result)
            
            # 처리 결과 요약
            if result["status"] == "success":
                print(f"✅ [{i+1}/{len(videos)}] 처리 성공!")
                print(f"🔗 Notion 페이지: {result['data']['notion_page']['page_url']}")
            else:
                print(f"❌ [{i+1}/{len(videos)}] 처리 실패: {result['message']}")
            
            # 다음 영상 처리 전 잠시 대기
            if i < len(videos) - 1:
                print(f"⏳ 다음 영상 처리를 위해 3초 대기...")
                time.sleep(3)
        
        # 전체 결과 요약
        print(f"\n🎉 모든 영상 처리 완료!")
        success_count = sum(1 for r in results if r["status"] == "success")
        print(f"📊 처리 결과: {success_count}/{len(videos)}개 성공")
        
        return results
        
    except Exception as e:
        logger.error(f"키워드 자동화 실패: {str(e)}")
        return [{"status": "error", "message": f"키워드 자동화 실패: {str(e)}"}]

In [ ]:
def process_single_video(url: str, keyword: str, video_info: Dict, video_index: int, custom_prompt: str = None) -> Dict[str, Any]:
    """단일 영상을 처리합니다."""
    
    try:
        # 자막 추출
        print(f"📄 자막 추출 중...")
        transcript_result = extract_youtube_transcript(url)
        
        if transcript_result["status"] != "success":
            return {
                "status": "error",
                "step": "transcript", 
                "message": transcript_result["message"]
            }
        
        transcript_data = transcript_result["data"]
        source_note = " (테스트)" if transcript_data.get("source") == "test_data" else ""
        print(f"   ✅ 자막 추출 성공: {transcript_data['length']} 문자{source_note}")
        
        # AI 요약
        print(f"🤖 AI 요약 생성 중...")
        summary_result = summarize_with_gemini(
            content=transcript_data["content"],
            keyword=keyword,
            custom_prompt=custom_prompt
        )
        
        if summary_result["status"] != "success":
            return {
                "status": "error",
                "step": "summary",
                "message": summary_result["message"]
            }
        
        summary_data = summary_result["data"]
        source_note = " (테스트)" if summary_data.get("source") == "test_data" else ""
        print(f"   ✅ AI 요약 성공: {summary_data['compression_ratio']}% 압축{source_note}")
        
        # Notion 페이지 생성
        print(f"📝 Notion 페이지 생성 중...")
        
        page_title = f"{keyword} 학습 자료 [{video_index}] - {video_info['title'][:30]}"
        
        notion_result = create_notion_page(
            title=page_title,
            summary=summary_data["summary"],
            url=url,
            keyword=keyword,
            transcript_data=transcript_data,
            summary_data=summary_data,
            video_info=video_info
        )
        
        if notion_result["status"] != "success":
            return {
                "status": "error",
                "step": "notion",
                "message": notion_result["message"]
            }
        
        source_note = " (테스트)" if notion_result["data"].get("source") == "test_data" else ""
        print(f"   ✅ Notion 페이지 생성 성공!{source_note}")
        
        return {
            "status": "success",
            "data": {
                "video_info": video_info,
                "transcript_info": {
                    "length": transcript_data["length"],
                    "source": transcript_data.get("source", "unknown")
                },
                "summary_info": {
                    "length": summary_data["summary_length"],
                    "compression_ratio": summary_data["compression_ratio"],
                    "source": summary_data.get("source", "unknown")
                },
                "notion_page": notion_result["data"]
            }
        }
        
    except Exception as e:
        return {"status": "error", "message": f"영상 처리 실패: {str(e)}"}

In [ ]:
# 💬 사용자 인터랙티브 함수들
def interactive_keyword_mode():
    """사용자가 키워드를 입력하여 자동화를 실행하는 인터랙티브 모드"""
    
    print(f"\n🍹 ConceptDrink 인터랙티브 모드")
    print(f"🎯 키워드를 입력하면 관련 YouTube 교육 영상을 자동으로 처리합니다!")
    print(f"="*60)
    
    while True:
        try:
            print(f"\n💬 사용 방법:")
            print(f"   • 학습하고 싶은 키워드 입력 (예: '파이썬', '자바', '머신러닝')")
            print(f"   • 'exit', 'quit', '종료' 입력시 프로그램 종료")
            
            user_input = input(f"\n🔍 검색할 키워드를 입력하세요: ").strip()
            
            # 종료 조건
            if user_input.lower() in ['exit', 'quit', '종료', 'bye']:
                print(f"👋 ConceptDrink를 종료합니다. 감사합니다!")
                break
            
            # 빈 입력 처리
            if not user_input:
                print(f"⚠️ 키워드를 입력해주세요.")
                continue
            
            # 영상 개수 설정
            print(f"\n📊 처리할 영상 개수를 선택하세요:")
            print(f"   1. 1개 (빠른 테스트)")
            print(f"   2. 3개 (추천)")
            print(f"   3. 5개 (상세 분석)")
            
            try:
                video_choice = input(f"선택 (1-3, 기본값: 2): ").strip()
                if video_choice == '1':
                    max_videos = 1
                elif video_choice == '3':
                    max_videos = 5
                else:
                    max_videos = 3
            except:
                max_videos = 3
            
            # AI와 대화하여 커스텀 프롬프트 생성
            print(f"\n🤖 AI와 대화하여 요약 스타일을 정해보세요!")
            print(f"💬 원하는 요약 방식을 자유롭게 말씀해주시면 AI가 최적의 프롬프트를 만들어드립니다.")
            print(f"📋 예시: '초보자도 이해하기 쉽게', '실무에서 바로 써먹을 수 있게', '마케팅 관점에서' 등")
            
            # 사용자 요청 입력
            user_request = input(f"\n💭 어떤 방식으로 요약하길 원하시나요? (엔터: 기본 요약): ").strip()
            
            custom_prompt = None
            if user_request:
                print(f"\n🤖 AI가 최적의 요약 프롬프트를 생성 중...")
                
                # AI와 대화하여 프롬프트 생성
                ai_conversation_result = create_custom_prompt_with_ai(user_input, user_request)
                
                if ai_conversation_result["status"] == "success":
                    suggested_prompt = ai_conversation_result["data"]["prompt"]
                    explanation = ai_conversation_result["data"]["explanation"]
                    
                    print(f"\n✨ AI가 제안하는 요약 방식:")
                    print(f"📝 {explanation}")
                    print(f"\n🎯 생성된 프롬프트:")
                    print(f"'{suggested_prompt[:100]}...'")
                    
                    # 사용자 확인
                    confirm = input(f"\n👍 이 방식으로 요약할까요? (y/n, 기본값: y): ").strip().lower()
                    
                    if confirm in ['', 'y', 'yes', '네', 'ㅇ']:
                        custom_prompt = suggested_prompt
                        print(f"✅ 커스텀 요약 방식이 설정되었습니다!")
                    else:
                        print(f"💡 기본 요약 방식을 사용합니다.")
                else:
                    print(f"⚠️ AI 프롬프트 생성 실패. 기본 요약을 사용합니다.")
            else:
                print(f"📋 기본 요약 방식을 사용합니다.")
            
            print(f"\n🚀 '{user_input}' 키워드로 {max_videos}개 영상을 처리합니다...")
            if custom_prompt:
                print(f"🤖 커스텀 요약 스타일: {custom_prompt[:50]}...")
            
            # 실제 처리 실행
            results = process_keyword_to_notion(user_input, max_videos, custom_prompt)
            
            # 결과 표시
            print(f"\n📋 처리 결과:")
            success_results = [r for r in results if r.get("status") == "success"]
            
            if success_results:
                print(f"✅ 성공: {len(success_results)}개")
                print(f"\n🔗 생성된 Notion 페이지들:")
                for i, result in enumerate(success_results):
                    page_url = result["data"]["notion_page"]["page_url"]
                    print(f"   {i+1}. {page_url}")
            
            failed_results = [r for r in results if r.get("status") != "success"]
            if failed_results:
                print(f"❌ 실패: {len(failed_results)}개")
            
            print(f"\n🎉 '{user_input}' 키워드 처리 완료!")
            
        except KeyboardInterrupt:
            print(f"\n👋 사용자에 의해 중단되었습니다.")
            break
        except Exception as e:
            print(f"❌ 오류 발생: {str(e)}")
            continue
def quick_keyword_process(keyword: str, max_videos: int = 3, custom_prompt: str = None):
    """키워드를 간편하게 처리하는 함수"""
    print(f"🍹 ConceptDrink 키워드 처리: '{keyword}'")
    
    results = process_keyword_to_notion(keyword, max_videos, custom_prompt)
    
    success_count = sum(1 for r in results if r.get("status") == "success")
    print(f"\n✅ 처리 완료: {success_count}/{len(results)}개 성공")
    
    return results

In [ ]:
def show_usage_guide():
    """상세한 사용법 가이드"""
    print(f"""
🍹 ConceptDrink 완전 사용 가이드
{"="*40}

🎯 주요 기능:
   1. 키워드 → YouTube 검색 → 자막 추출 → AI 요약 → Notion 저장
   2. 사용자 커스텀 요약 스타일 지원 (초보자용, 실무용, 마케팅용 등)

🚀 사용 방법:

1️⃣ 인터랙티브 모드 (추천!):
   interactive_keyword_mode()
   
   → 키워드를 입력하면 자동으로 관련 교육 영상들을 처리

2️⃣ 간편 처리:
   quick_keyword_process("파이썬")
   quick_keyword_process("자바", 5)  # 5개 영상 처리

🔧 설정 필요사항:

필수 환경변수:
   • GEMINI_API_KEY: Google AI Studio에서 발급
   • NOTION_TOKEN: Notion 개발자 설정에서 발급  
   • NOTION_PARENT_PAGE: Notion 페이지 ID

선택 환경변수:
   • YOUTUBE_API_KEY: YouTube Data API (검색 기능 향상)

📚 키워드 예시:
   • 프로그래밍: "파이썬", "자바", "자바스크립트"
   • 기술: "머신러닝", "AI", "블록체인", "클라우드"
   • 기타: "영어회화", "마케팅", "디자인"

💡 팁:
   • 구체적인 키워드일수록 더 정확한 결과
   • 한 번에 3-5개 영상 처리 추천

🎉 목표: 학습하고 싶은 주제의 키워드만 입력하면
       관련 YouTube 교육 영상들이 자동으로 Notion에 요약되어 저장!
""")

In [ ]:
# 🚀 실제 사용을 위한 메인 실행 블록
if __name__ == "__main__":
    print(f"\n{'='*60}")
    print(f"🍹 ConceptDrink YouTube 키워드 검색 & 자동화 시스템")
    print(f"{'='*60}")
    
    if core_apis_ready:
        print(f"\n🎉 시스템 준비 완료!")
        
        # 사용법 가이드
        show_usage_guide()
        
        # 🔧 수정: 실제 사용자 입력 받기
        print(f"\n🚀 인터랙티브 모드를 시작하시겠습니까? (y/n): ", end="")
        
        try:
            user_choice = input().strip().lower()
            
            if user_choice in ['y', 'yes', '네', 'ㅇ', '']:
                print(f"📋 인터랙티브 모드를 시작합니다...")
                
                # 인터랙티브 모드 실행
                interactive_keyword_mode()
            else:
                print(f"💡 수동 모드입니다. 다음 함수들을 사용하여 직접 실행하세요:")
                print(f"   • quick_keyword_process('파이썬')")  
                print(f"   • interactive_keyword_mode()")
                
        except KeyboardInterrupt:
            print(f"\n👋 프로그램을 종료합니다.")
        
    else:
        print(f"\n⚠️ API 키 설정이 필요합니다!")
        print(f"📝 .env 파일에 다음 키들을 설정해주세요:")
        print(f"   • GEMINI_API_KEY=your_key")
        print(f"   • NOTION_TOKEN=your_token")
        print(f"   • NOTION_PARENT_PAGE=your_page_id")
        
    print(f"\n{'='*60}")

print("✅ ConceptDrink 실제 사용 버전 로드 완료!")

우회방법

In [ ]:
# 🚀 Gemini 2.5 모델로 재시도! (ADK 완전 우회!)\n

print("🚀 Gemini 2.5 모델로 재시도!")
print("=" * 40)

import google.generativeai as genai

try:
    # API 키 설정\n
    genai.configure(api_key=GEMINI_API_KEY)
    print("✅ API 키 설정 완료")
    
    # 실제 사용 가능한 새 모델들로 시도\n
    new_models = [
        'models/gemini-2.5-flash',                    # 가장 최신, 빠름\n
        'models/gemini-2.5-pro-preview-03-25',       # 고성능\n
        'gemini-1.5-pro',                            # 기존 모델\n
        'gemini-1.5-flash'                           # 기존 모델\n
    ]
    model = None
    working_model_name = None
    
    for model_name in new_models:
        try:
            print(f"🧪 {model_name} 테스트 중...")
            
            # timeout 제거하고 테스트\n
            test_model = genai.GenerativeModel(model_name)
            response = test_model.generate_content("안녕하세요")
            
            if response and response.text:
                print(f"   ✅ {model_name} 작동 성공!")
                print(f"   📝 테스트 응답: {response.text[:50]}...")
                model = test_model
                working_model_name = model_name
                break
                
        except Exception as e:
            print(f"   ❌ {model_name} 실패: {str(e)[:50]}...")
            continue
    
    if model and working_model_name:
        print(f"\\n🎉 성공! 작동하는 모델: {working_model_name}")
        
        # 완전한 대화 시스템 생성\n
        def chat_working(message: str) -> str:
            """작동하는 Gemini로 대화"""
            try:
                prompt = f"""
당신은 YouTube 처리 전문 AI Assistant입니다.

주요 기능:
- YouTube 영상 검색 및 자막 추출
- AI를 이용한 컨텐츠 요약
- Notion 페이지 자동 생성
- 일반적인 대화 및 도움

사용자 질문: {message}

친근하고 도움이 되는 한국어로 답변해주세요. 이모지를 적절히 사용하세요."""
                
                response = model.generate_content(
                    prompt,
                    generation_config=genai.types.GenerationConfig(
                        max_output_tokens=3000,
                        temperature=0.7
                    )
                )
                
                return response.text if response and response.text else "응답을 생성하지 못했습니다."
                
            except Exception as e:
                return f"❌ 응답 오류: {str(e)}"
        
        # 사용하기 쉬운 함수들\n
        def ask_ai_working(question: str):
            """작동하는 AI 질문 함수"""
            print(f"💬 질문: {question}")
            print(f"🤖 AI가 답변 생성 중... (모델: {working_model_name})")
            
            response = chat_working(question)
            print(f"🤖 AI 답변: {response}")
            print("-" * 60)
            return response
        
        def interactive_chat_working():
            """작동하는 대화형 모드"""
            print(f"💬 Gemini 대화 모드 시작!")
            print(f"🤖 사용 모델: {working_model_name}")
            print(f"🎯 YouTube 관련 질문이나 일반 대화 모두 가능!")
            print(f"📋 종료하려면 'exit' 입력")
            print("=" * 60)
            
            while True:
                try:
                    user_input = input("\\n💬 질문: ").strip()
                    
                    if user_input.lower() in ['exit', 'quit', 'bye']:
                        print("👋 대화를 종료합니다.")
                        break
                    
                    if not user_input:
                        continue
                    
                    print("🤖 Gemini가 답변 중...")
                    response = chat_working(user_input)
                    print(f"\\n🤖 AI 답변:\\n{response}\\n")
                    
                except KeyboardInterrupt:
                    print("\\n👋 사용자에 의해 중단되었습니다.")
                    break
                except Exception as e:
                    print(f"❌ 오류 발생: {e}")
        
        # 전역 함수 교체 (ADK 대신 작동하는 버전으로!)
        globals()['ask_ai'] = ask_ai_working
        globals()['interactive_chat'] = interactive_chat_working
        globals()['chat_working'] = chat_working
        
        # 즉시 테스트!\n
        print(f"\\n🧪 최종 테스트:")
        test_result = ask_ai_working("안녕하세요! 이제 정말 작동하나요?")
        
        if test_result and "❌" not in test_result and len(test_result) > 10:
            print(f"\\n🎉🎉🎉 드디어 성공!!! 🎉🎉🎉")
            print(f"\\n🚀 이제 완전히 작동합니다:")
            print(f"   • ask_ai('질문') - 일반 대화")
            print(f"   • interactive_chat() - 대화형 모드")
            print(f"\\n🔥 ADK Runner 문제를 완전히 우회했습니다!")
            
        else:
            print(f"\\n⚠️ 여전히 문제가 있습니다: {test_result}")
        
    else:
        print(f"\\n❌ 모든 모델 테스트 실패")
        
except Exception as e:
    print(f"❌ 실패: {str(e)}")

print(f"\\n✅ 설정 완료!")